In [4]:
# import libaries 
import kagglehub
import os
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler)
from sklearn.compose import ColumnTransformer
# import UFC stats (individual figthers)
# Download latest version
data_path_set = '/Users/samuelcuriel/.cache/kagglehub/datasets/asaniczka/ufc-fighters-statistics/versions/16'
# Load the data from the csv and read it 
# to a dataframe
files = os.listdir(data_path_set)

file_path = os.path.join(data_path_set, 'ufc-fighters-statistics.csv')
df = pd.read_csv(file_path)


# import UFC fights (figther a v fighter b)
# Download latest version

data_path_fights = '/Users/samuelcuriel/.cache/kagglehub/datasets/mdabbert/ultimate-ufc-dataset/versions/177'
files2 = os.listdir(data_path_fights)
print(files2)
# load the data from the csv and read it
# to a datframe

fight_path = os.path.join(data_path_fights,'ufc-master.csv')
df_fights = pd.read_csv(fight_path)


StandardScaler()
['upcoming.csv', 'ufc-master.csv']
                name    nickname  wins  losses  draws  height_cm  \
0    Robert Drysdale         NaN     7       0      0     190.50   
1  Daniel McWilliams  The Animal    15      37      0     185.42   
2         Dan Molina         NaN    13       9      0     177.80   
3          Paul Ruiz         NaN     7       4      0     167.64   
4    Collin Huckbody      All In     8       2      0     190.50   

   weight_in_kg  reach_in_cm    stance date_of_birth  \
0         92.99          NaN  Orthodox    1981-10-05   
1         83.91          NaN       NaN           NaN   
2         97.98          NaN       NaN           NaN   
3         61.23          NaN       NaN           NaN   
4         83.91       193.04  Orthodox    1994-09-29   

   significant_strikes_landed_per_minute  significant_striking_accuracy  \
0                                   0.00                            0.0   
1                                   3.36            

In [13]:
# Merge both datasets
merged_df = df_fights.merge(df, left_on="RedFighter", right_on="name", suffixes=("_Red", "_drop"))
merged_df = merged_df.merge(df, left_on="BlueFighter", right_on="name", suffixes=("_Red", "_Blue"))

# Feature engineering to choose the red and blue fighters differences 
merged_df["height_diff"] = merged_df["height_cm_Red"] - merged_df["height_cm_Blue"]
merged_df["reach_diff"] = merged_df["reach_in_cm_Red"] - merged_df["reach_in_cm_Blue"]
merged_df["win_ratio_diff"] = (merged_df["wins_Red"] / (merged_df["wins_Red"] + merged_df["losses_Red"])) - \
                              (merged_df["wins_Blue"] / (merged_df["wins_Blue"] + merged_df["losses_Blue"]))
merged_df["striking_diff"] = merged_df["significant_strikes_landed_per_minute_Red"] - merged_df["significant_strikes_landed_per_minute_Blue"]
merged_df["takedown_diff"] = merged_df["average_takedowns_landed_per_15_minutes_Red"] - merged_df["average_takedowns_landed_per_15_minutes_Blue"]

# Now we have a dataset to work with and next cell we create pipeline!

       RedFighter       BlueFighter  RedOdds  BlueOdds  RedExpectedValue  \
0      Ciryl Gane  Alexander Volkov   -380.0     300.0           26.3158   
1  Bryce Mitchell       Kron Gracie   -950.0     625.0           10.5263   
2   Nate Landwehr        Dooho Choi   -130.0     110.0           76.9231   
3  Dominick Reyes     Anthony Smith   -380.0     300.0           26.3158   
4   Vicente Luque    Themba Gorimbo   -650.0    -162.0           15.3846   

   BlueExpectedValue        Date                Location Country Winner  ...  \
0           300.0000  2024-12-07  Las Vegas, Nevada, USA     USA    Red  ...   
1           625.0000  2024-12-07  Las Vegas, Nevada, USA     USA    Red  ...   
2           110.0000  2024-12-07  Las Vegas, Nevada, USA     USA   Blue  ...   
3           300.0000  2024-12-07  Las Vegas, Nevada, USA     USA    Red  ...   
4            61.7284  2024-12-07  Las Vegas, Nevada, USA     USA    Red  ...   

   significant_strike_defence_Blue  \
0                       